In [1]:
import pyspark
import random
from pyspark.sql import SparkSession
spark = SparkSession\
.builder\
.appName("Tema2")\
.getOrCreate()

In [2]:
mi_rango = spark.range(1000).toDF("numero")

In [3]:
divisoresDe2 = mi_rango.where("numero % 2 = 0")

In [4]:
divisoresDe2.count()

500

In [5]:
mnm_df = (spark.read.format("csv")
.option("header", "true")
.option("inferSchema", "true")
.load("C:\Spark\data\mnm_dataset.csv"))

In [7]:
mnm_df.take(3)

[Row(State='TX', Color='Red', Count=20),
 Row(State='NV', Color='Blue', Count=66),
 Row(State='CO', Color='Blue', Count=79)]

In [8]:
mnm_df.sort("Color").explain()

== Physical Plan ==
*(1) Sort [Color#27 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(Color#27 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#57]
   +- FileScan csv [State#26,Color#27,Count#28] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/C:/Spark/data/mnm_dataset.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<State:string,Color:string,Count:int>




In [9]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

In [12]:
mnm_df.sort("Color").explain()

== Physical Plan ==
*(1) Sort [Color#27 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(Color#27 ASC NULLS FIRST, 5), ENSURE_REQUIREMENTS, [id=#69]
   +- FileScan csv [State#26,Color#27,Count#28] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/C:/Spark/data/mnm_dataset.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<State:string,Color:string,Count:int>




In [13]:
mnm_df.sort("Color").take(3)

[Row(State='OR', Color='Blue', Count=71),
 Row(State='CO', Color='Blue', Count=79),
 Row(State='NV', Color='Blue', Count=66)]

In [14]:
mnm_df.createOrReplaceTempView("mnms")

In [21]:
count_mnm = spark.sql("SELECT STATE, COLOR, COUNT(*) FROM mnms GROUP BY STATE, COLOR ORDER BY COUNT(*)")

In [22]:
count_mnm.take(5)

[Row(STATE='WY', COLOR='Brown', count(1)=1532),
 Row(STATE='UT', COLOR='Green', count(1)=1591),
 Row(STATE='WY', COLOR='Orange', count(1)=1595),
 Row(STATE='CA', COLOR='Blue', count(1)=1603),
 Row(STATE='NV', COLOR='Red', count(1)=1610)]

In [23]:
ca_count_mnm = spark.sql("SELECT STATE, COLOR, COUNT(*) FROM mnms WHERE STATE = 'CA' GROUP BY STATE, COLOR ORDER BY COUNT(*)")

In [24]:
ca_count_mnm.take(6)

[Row(STATE='CA', COLOR='Blue', count(1)=1603),
 Row(STATE='CA', COLOR='Red', count(1)=1656),
 Row(STATE='CA', COLOR='Orange', count(1)=1657),
 Row(STATE='CA', COLOR='Brown', count(1)=1718),
 Row(STATE='CA', COLOR='Green', count(1)=1723),
 Row(STATE='CA', COLOR='Yellow', count(1)=1807)]

In [25]:
strings = spark.read.textFile("C:\Users\GAME\Desktop\el_quijote.txt")

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-25-52c52b88e870>, line 1)

In [27]:
strings = spark.read.text("C:\Users\GAME\Desktop\el_quijote.txt")

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-27-9a2b1a47e6c7>, line 1)

In [38]:
max_mnm = spark.sql("SELECT STATE, COLOR, MAX(Count) FROM mnms GROUP BY STATE, COLOR ORDER BY MAX(Count) asc")

In [36]:
max_mnm.take(100)

[Row(STATE='WA', COLOR='Yellow', max(Count)=100),
 Row(STATE='NV', COLOR='Red', max(Count)=100),
 Row(STATE='AZ', COLOR='Green', max(Count)=100),
 Row(STATE='NM', COLOR='Yellow', max(Count)=100),
 Row(STATE='TX', COLOR='Orange', max(Count)=100),
 Row(STATE='NV', COLOR='Orange', max(Count)=100),
 Row(STATE='CA', COLOR='Red', max(Count)=100),
 Row(STATE='TX', COLOR='Yellow', max(Count)=100),
 Row(STATE='CO', COLOR='Brown', max(Count)=100),
 Row(STATE='NM', COLOR='Brown', max(Count)=100),
 Row(STATE='WY', COLOR='Brown', max(Count)=100),
 Row(STATE='AZ', COLOR='Yellow', max(Count)=100),
 Row(STATE='CO', COLOR='Blue', max(Count)=100),
 Row(STATE='OR', COLOR='Blue', max(Count)=100),
 Row(STATE='CA', COLOR='Yellow', max(Count)=100),
 Row(STATE='TX', COLOR='Green', max(Count)=100),
 Row(STATE='WY', COLOR='Orange', max(Count)=100),
 Row(STATE='CO', COLOR='Red', max(Count)=100),
 Row(STATE='CA', COLOR='Green', max(Count)=100),
 Row(STATE='WA', COLOR='Orange', max(Count)=100),
 Row(STATE='CA', CO

In [39]:
ca_count_mnm = spark.sql("SELECT STATE, COLOR, COUNT(*) FROM mnms WHERE STATE = 'CA' OR STATE = 'TX' GROUP BY STATE, COLOR ORDER BY COUNT(*)")

In [40]:
ca_count_mnm.take(5)

[Row(STATE='CA', COLOR='Blue', count(1)=1603),
 Row(STATE='TX', COLOR='Blue', count(1)=1614),
 Row(STATE='TX', COLOR='Brown', count(1)=1641),
 Row(STATE='TX', COLOR='Orange', count(1)=1652),
 Row(STATE='CA', COLOR='Red', count(1)=1656)]

In [47]:
count_mnm_df = (mnm_df\
.select("State", "Color", "Count")\
.groupBy("State", "Color")\
.agg(max("Count").alias("Total"))\
.orderBy("Total", ascending=False))

from pyspark.sql.functions import max

In [46]:
from pyspark.sql.functions import max

In [48]:
count_mnm_df.take(10)

[Row(State='WY', Color='Blue', Total=100),
 Row(State='AZ', Color='Blue', Total=100),
 Row(State='OR', Color='Brown', Total=100),
 Row(State='WA', Color='Red', Total=100),
 Row(State='WY', Color='Red', Total=100),
 Row(State='WY', Color='Yellow', Total=100),
 Row(State='AZ', Color='Orange', Total=100),
 Row(State='AZ', Color='Red', Total=100),
 Row(State='NV', Color='Yellow', Total=100),
 Row(State='WA', Color='Brown', Total=100)]

In [56]:
ca_count_mnm_df = (mnm_df\
.select("State", "Color", "Count")\
.where("mnm_df.State == 'CA' or mnm_df.State == 'TX'")\
.groupBy("State", "Color")\
.agg(count("Count").alias("Total"))\
.orderBy("Total", ascending=False))

SyntaxError: invalid syntax (<ipython-input-56-a10916f937e3>, line 3)